In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()
hotels

{'html_attributions': [],
 'next_page_token': 'CqQCHAEAAPsMm3bJZqDE-KBS9tVVUgzV9ShPQt6oJfNsA7VM1DwiP25wTUjvraqefjNpOGSI7BrslnLyny0GDCjY_BVXwKM3sXv9zTNQC2BcxpxmRpagoMm7tRF9Hq2DdfmJ3q-wl-V2z991Xpb-Pwd3-TA31N5_N6RDsVUwzi0wmrwxHL0yyeL3f3lOx4OFR0XvkHUbR-Khq_OejQ9B0rcqYG9ujvE-OPPebZkIn1AllIRMoSTBcKfdJJi9yomuHb-Hxqqha-SjqajOxWlZMCUTMCq4ijFiSkUP76lKOdQkz79XUI3GGUzzVvTQhLjX272Qr2v5XoAUKaMOPnliQq_REphEl-n73I4ipxOCxXauyp18iwGZjymqrrrOo_mO9-WjkXzAPxIQW385jkUoeYomJJ8pKh-vXxoUi10mId_gQ7u9hHH3UsqL4DBrzEs',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594402302915,
      'lng': 2.354348530291502},
     'southwest': {'lat': 48.8567422697085, 'lng': 2.351650569708498}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': '7fa47cb61d7345cecc3b802d0e04fcff2b3d5ecb',
   'name': 'Hôtel Duo',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 3840,

In [3]:
len(hotels["results"])

20

In [4]:
hotels["results"][0]["name"]

'Hôtel Duo'

In [5]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Andra,RU,2020-04-22 05:37:35,62.51,65.89,43.05,88,100,13.85
1,1,Plouzane,FR,2020-04-22 05:37:35,48.38,-4.62,48.20,93,95,8.05
2,2,Mataura,NZ,2020-04-22 05:37:36,-46.19,168.86,54.00,80,100,1.01
3,3,Rikitea,PF,2020-04-22 05:37:36,-23.12,-134.97,75.81,79,100,16.35
4,4,Bhanpura,IN,2020-04-22 05:37:36,24.51,75.75,95.25,16,16,2.62


In [6]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= 90) & \
                                       (city_data_df["Max Temp"] >= 75)]
preferred_cities_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
3,3,Rikitea,PF,2020-04-22 05:37:36,-23.12,-134.97,75.81,79,100,16.35
5,5,Barra Patuca,HN,2020-04-22 05:37:36,15.80,-84.28,79.83,82,14,13.18
7,7,Avarua,CK,2020-04-22 05:37:37,-21.21,-159.78,78.80,78,1,9.17
9,9,Carnarvon,AU,2020-04-22 05:37:37,-24.87,113.63,78.80,69,1,16.11
20,20,Kavieng,PG,2020-04-22 05:37:39,-2.57,150.80,83.73,74,79,10.27


In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head

<bound method NDFrame.head of              City Country  Max Temp    Lat     Lng Hotel Name
3         Rikitea      PF     75.81 -23.12 -134.97           
5    Barra Patuca      HN     79.83  15.80  -84.28           
7          Avarua      CK     78.80 -21.21 -159.78           
9       Carnarvon      AU     78.80 -24.87  113.63           
20        Kavieng      PG     83.73  -2.57  150.80           
..            ...     ...       ...    ...     ...        ...
533        Bauchi      NG     78.35  10.50   10.00           
538          Lere      NG     76.96  10.39    8.57           
542         Birao      CF     84.69  10.28   22.79           
547          Doha      QA     80.60  25.29   51.53           
558        Kollam      IN     89.60   8.88   76.60           

[141 rows x 6 columns]>

In [8]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")


In [9]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Add pop up markers
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))